In [2]:
import os

from src.envs.utils import (
                            RandomErdosRenyiGraphGenerator,
                            RandomBarabasiAlbertGraphGenerator,
                            RandomCompleteGraphGenerator,
                            EdgeType,
                            )
from scipy.sparse import csr_matrix,save_npz,load_npz
from experiments.utils import load_graph_set, mk_dir

In [2]:
number_of_vertices = [20,40,60,100,200]
distributions = ['ER','BA']
number_of_instances=4000

for distribution in distributions:
    for n in number_of_vertices:
        folder=f'data/training/{distribution}_{n}'
        mk_dir(folder)
        
        if distribution=='ER':

            train_graph_generator = RandomErdosRenyiGraphGenerator(n_spins=n,
                                                                    p_connection=0.15,
                                                                    edge_type=EdgeType.DISCRETE)
        else:
            train_graph_generator = RandomBarabasiAlbertGraphGenerator(n_spins= n,
                                                                       m_insertion_edges=4,
                                                                       edge_type=EdgeType.DISCRETE)

        for i in range(number_of_instances):
            graph=train_graph_generator.get()
            save_file_name = f'{distribution}_{n}vertices_graph_{i}'
            save_file_path=os.path.join(folder,save_file_name)
            sparse_matrix = csr_matrix(graph)
            
            save_npz(save_file_path, sparse_matrix)
        


dir already exists:  data/training/ER_20
dir already exists:  data/training/ER_40
dir already exists:  data/training/ER_60
dir already exists:  data/training/ER_100
dir already exists:  data/training/ER_200
dir already exists:  data/training/BA_20
dir already exists:  data/training/BA_40
dir already exists:  data/training/BA_60
dir already exists:  data/training/BA_100
dir already exists:  data/training/BA_200


In [11]:
import shutil
import pickle
import pandas as pd

number_of_vertices = [20,40,60,100,200]
distributions = ['ER','BA']
for distribution in distributions:
    for n in number_of_vertices:
        test_folder=f'../data/testing/{distribution}_{n}'
        val_folder=f'../data/validation/{distribution}_{n}'
        shutil.rmtree(test_folder)
        shutil.rmtree(val_folder)
        # os.rmdir(test_folder)
        # os.rmdir(val_folder)
        mk_dir(test_folder)
        mk_dir(val_folder)
        
        if distribution=='ER':
            val_graph_save_loc=f'_graphs/testing/ER_{n}spin_p15_50graphs.pkl'
            test_graph_save_loc=f'_graphs/validation/ER_{n}spin_p15_100graphs.pkl'
            opt_test=pickle.load(open(f'_graphs/validation/opts/cuts_ER_{n}spin_p15_100graphs.pkl', 'rb'))
        elif distribution=='BA':
            val_graph_save_loc=f"_graphs/testing/BA_{n}spin_m4_50graphs.pkl"
            test_graph_save_loc=f"_graphs/validation/BA_{n}spin_m4_100graphs.pkl"
            opt_test=pickle.load(open(f'_graphs/validation/opts/cuts_BA_{n}spin_m4_100graphs.pkl', 'rb'))
            
        graphs_test = load_graph_set(test_graph_save_loc)
        graphs_val = load_graph_set(val_graph_save_loc)

        test_files=[]
        
        for i,graph in enumerate(graphs_test):
            save_file_name = f'{distribution}_{n}vertices_graph_{str(i).zfill(4)}'
            save_file_path=os.path.join(test_folder,save_file_name)
            sparse_matrix = csr_matrix(graph)
            save_npz(save_file_path, sparse_matrix)
            test_files.append(save_file_name)

        
        opt_file={'Instance':test_files,'OPT':opt_test}
        df=pd.DataFrame(opt_file)
        df.to_pickle('optimal')

        for i,graph in enumerate(graphs_val):
            save_file_name = f'{distribution}_{n}vertices_graph_{str(i).zfill(4)}'
            save_file_path=os.path.join(val_folder,save_file_name)
            sparse_matrix = csr_matrix(graph)
            save_npz(save_file_path, sparse_matrix)
            

created dir:  ../data/testing/ER_20
created dir:  ../data/validation/ER_20
100 target graphs loaded from _graphs/validation/ER_20spin_p15_100graphs.pkl
50 target graphs loaded from _graphs/testing/ER_20spin_p15_50graphs.pkl
created dir:  ../data/testing/ER_40
created dir:  ../data/validation/ER_40
100 target graphs loaded from _graphs/validation/ER_40spin_p15_100graphs.pkl
50 target graphs loaded from _graphs/testing/ER_40spin_p15_50graphs.pkl
created dir:  ../data/testing/ER_60
created dir:  ../data/validation/ER_60
100 target graphs loaded from _graphs/validation/ER_60spin_p15_100graphs.pkl
50 target graphs loaded from _graphs/testing/ER_60spin_p15_50graphs.pkl
created dir:  ../data/testing/ER_100
created dir:  ../data/validation/ER_100
100 target graphs loaded from _graphs/validation/ER_100spin_p15_100graphs.pkl
50 target graphs loaded from _graphs/testing/ER_100spin_p15_50graphs.pkl
created dir:  ../data/testing/ER_200
created dir:  ../data/validation/ER_200
100 target graphs loade

In [12]:
df

,Instance,OPT
0,BA_200vertices_graph_0000,184.0
1,BA_200vertices_graph_0001,192.0
2,BA_200vertices_graph_0002,184.0
3,BA_200vertices_graph_0003,198.0
4,BA_200vertices_graph_0004,194.0
...,...,...
95,BA_200vertices_graph_0095,187.0
96,BA_200vertices_graph_0096,202.0
97,BA_200vertices_graph_0097,184.0
98,BA_200vertices_graph_0098,194.0
